In [5]:
import qiskit

from PatchedMeasCal.edge_bfs import CouplingMapGraph
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid, Hexagonal, FullyConnected, SquareOctagonal, FakeBackendWrapper



from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

import qiskit.tools.jupyter

In [3]:
Hexagonal(3, 1)

<Hexagonal('hex')>

In [25]:
class Hexagonal16(FakeBackendWrapper):
    """A fake 16 qubit fully connected backend."""

    def __init__(self):
        self.n_qubits = 16
        self._coupling_map = self.gen_coupling_map()
        super().__init__("hex", self.n_qubits, self._coupling_map)

    
    def gen_coupling_map(self):
        return [
        [0, 1], [1, 0],
        [1, 2], [2, 1],
        [2, 3], [3, 2],
        [3, 4], [4, 3],
        [4, 5], [5, 4],
        [0, 5], [5, 0],
        [5, 6], [6, 5],
        [6, 7], [7, 6],
        [7, 8], [8, 7],
        [8, 9], [9, 8],
        [9, 4], [4, 9],
        [3, 10], [10, 3],
        [10, 11], [11, 10],
        [11, 12], [12, 11],
        [12, 9], [9, 12],
        [0, 14], [14, 0],
        [14, 15], [15, 14],
        [15, 6], [6, 15]
    ]
Hexagonal16()

<Hexagonal16('hex')>

In [13]:
patches = CouplingMapGraph(dev.configuration().coupling_map)
patches.edge_patches()

NameError: name 'dev' is not defined

In [2]:
def plus_state_prep(backend):
    n_qubits = backend.n_qubits
    coupling_map = backend._coupling_map
    
    targeted_qubits = [0]
    circuit = qiskit.QuantumCircuit(n_qubits, n_qubits)
    circuit.h(0)
    
    while len(targeted_qubits) < n_qubits:
        for i in range(n_qubits):
            if i not in targeted_qubits: # Qubit not yet added
                for edge in coupling_map: # For each edge in the map
                    if i in edge:
                        ctrl = edge[edge[0] == i]
                        targ = edge[edge[1] == i]
                        circuit.cnot(ctrl, targ)
                        targeted_qubits.append(i)
                        break
    circuit.measure(list(range(n_qubits)), list(range(n_qubits)))
    return circuit  

In [3]:
Grid(4, 4)

<Grid('square')>

In [4]:
n_qubits = 16
backends = [Grid(4, 4)]#, Hexagonal16()]#, FullyConnected(n_qubits)]

n_shots = 16000

fitters = [TensorPatchFitter(backend, n_shots=n_shots) for backend in backends]
[fitter.build(verbose=True) for fitter in fitters]

Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [==================> ] 100.0% 
	Building Edge Patches : [===================>] 100.0% 
	Constructing Calibration Circuits : [================>   ] 100.0% 
	Executing Calibration Circuits
	De-hexing Measurement Results
	Tracing Patched Calibration Results : [================>   ] 100.0% 
Building Patch Calibrations
	Merging Patches : [=================>  ] 100.0% 
Building Measure Fitter
	Building Meas Fitters from Patches : [==================> ] 100.0% 


[None]

In [5]:
initial_layout = list(range(n_qubits))

for backend in backends[:1]:
    circ = plus_state_prep(backend)
    tc = qiskit.transpile(circ, backend=backend, initial_layout=initial_layout, optimization_level=0)
    results = qiskit.execute(tc, backend, shots=n_shots, initial_layout=initial_layout, optimization_level=0).result()

In [17]:
results.get_counts()['1' * 16]

2872

In [7]:
n_shots

16000

In [8]:
f_res = fitters[0].apply(results.get_counts())

In [18]:
f_res['1' * 16]

6283.463488538778